In [ ]:
# cell 1: Install & Setup (This takes about 2-3 mins)
print("🚀 Installing Unsloth (This makes training 2x faster)...")
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

from unsloth import FastLanguageModel
import torch

# 1. Configuration
max_seq_length = 2048
dtype = None
load_in_4bit = True

# 2. Load the Base Model (Llama 3 8B)
print("📥 Downloading Llama-3-8B Base Model...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # We use the 4-bit version to fit in free Colab
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 3. Add LoRA Adapters (This is the "Brain Surgery" part)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

# 4. Load YOUR Dataset
print("📚 Loading your Hindi Slang Dataset...")
from datasets import load_dataset

# Note: We are pointing specifically to YOUR repository and file
dataset = load_dataset("defnotutkarsh/hindi-slang-v1", data_files="final_train.json", split="train")

# 5. Format the Data for Llama-3
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

# 6. Start Training!
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

print("🔥 Starting Training... (This will take 30-60 mins)")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 300, # <-- QUICK RUN. Change to 200-300 for a full serious train.
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()
print("✅ Training Finished!")

🚀 Installing Unsloth (This makes training 2x faster)...
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-4s5evluf/unsloth_cec8e840a43945fe919b037e961ae51d
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-4s5evluf/unsloth_cec8e840a43945fe919b037e961ae51d
  Resolved https://github.com/unslothai/unsloth.git to commit 2eb6b0d5f363a60ed3792ea1f04250537ac66939
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached xformers-0.0.26.post1.tar.gz (4.1 MB)
  Preparing metadata (setup.py) ... done
  Using cached trl-0.8.6-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.8.6-py3-none-any.whl (245 kB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a prob

Unsloth 2025.12.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


📚 Loading your Hindi Slang Dataset...


Map:   0%|          | 0/851 [00:00<?, ? examples/s]

🔥 Starting Training... (This will take 30-60 mins)


...

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.708500
2,1.590000
3,1.630300
4,1.509600
5,1.368600
6,1.180500
7,0.872300
8,0.715700
9,0.662800
10,0.543000


wandb: WARNING URL not available in offline run


train/epoch,▁▁▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
train/global_step,▁▁▁▁▁▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇█████
train/grad_norm,▆▇█▃▄▃▂▂▃▄▃▂▂▃▂▃▂▃▃▂▃▂▂▃▃▃▃▂▁▃▂▂▃▃▂▃▂▂▂▂
train/learning_rate,██████▇▇▇▇▇▆▆▆▆▆▆▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁
train/loss,█▆▅▅▆▄▅▄▅▃▅▄▃▄▄▄▄▂▃▄▃▃▃▃▃▂▃▂▃▃▂▂▂▁▂▁▂▁▁▂
total_flos,1.5835652895031296e+16
train/epoch,2.80751
train/global_step,300
train/grad_norm,0.44618
train/learning_rate,0.0
train/loss,0.1867


✅ Training Finished!


In [ ]:
# Cell 3: SAVE THE V2 MODEL (Run this AFTER training finishes)
from google.colab import userdata

# 1. Paste your Write Token here
hf_token = "HIDDEN_TOKEN" # <--- REPLACE WITH YOUR TOKEN

# 2. V2 Name (The "Smarter" Version)
username = "defnotutkarsh"
model_name = "hindi-llama-3-slang-v2" # <--- Note the V2

print(f"💾 Saving V2 model to {username}/{model_name}...")

# 3. Push to Hub
model.push_to_hub(f"{username}/{model_name}", token=hf_token)
tokenizer.push_to_hub(f"{username}/{model_name}", token=hf_token)

print(f"✅ Saved! The V2 model is live: https://huggingface.co/{username}/{model_name}")

💾 Saving V2 model to defnotutkarsh/hindi-llama-3-slang-v2...


README.md:   0%|          | 0.00/580 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          |  556kB /  168MB            

Saved model to https://huggingface.co/defnotutkarsh/hindi-llama-3-slang-v2


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpltctz3wt/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

✅ Saved! The V2 model is live: https://huggingface.co/defnotutkarsh/hindi-llama-3-slang-v2


In [ ]:
# Cell 2: Inference (Testing the Model)
from unsloth import FastLanguageModel

# 1. Prepare for Chatting
FastLanguageModel.for_inference(model)

# 2. Your Question (You can change this!)
hindi_prompt = "Bhai, software engineer banne ke liye kya roadmap follow karu?"

# 3. Format strictly as Alpaca
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# 4. Generate Answer
inputs = tokenizer(
[
    alpaca_prompt.format(
        hindi_prompt, # The User Question
        "",           # Input (empty)
        "",           # Output (empty)
    )
], return_tensors = "pt").to("cuda")

print("🤔 AI is thinking...")
outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
result = tokenizer.batch_decode(outputs)[0]

# 5. Clean up the output to just show the answer
print("\n" + result.split("### Response:")[-1].replace("<|end_of_text|>", "").strip())


🤔 AI is thinking...

Bhai, software engineer banne ke liye, aap ek computer science ki degree ya diploma le sakte hain, fir ek internship ya training program join kar sakte hain, aur fir ek company join kar sakte hain, jahan aap software engineer ke roop mein kaam kar sakte hain.


In [ ]:
# Cell 3: Save to Hugging Face (PERMANENTLY)
from google.colab import userdata

# 1. PASTE YOUR WRITE TOKEN HERE
# Get it from: https://huggingface.co/settings/tokens
hf_token = "HIDDEN_TOKEN" # <--- REPLACE THIS WITH YOUR ACTUAL TOKEN

# 2. Your Identity & Project Name
username = "defnotutkarsh"
model_name = "hindi-llama-3-slang-v1"

print(f"💾 Saving model to {username}/{model_name}...")

# 3. Push to Hub
model.push_to_hub(f"{username}/{model_name}", token=hf_token)
tokenizer.push_to_hub(f"{username}/{model_name}", token=hf_token)

print(f"✅ Saved! View your model here: https://huggingface.co/{username}/{model_name}")


💾 Saving model to defnotutkarsh/hindi-llama-3-slang-v1...


README.md:   0%|          | 0.00/580 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          |  560kB /  168MB            

Saved model to https://huggingface.co/defnotutkarsh/hindi-llama-3-slang-v1


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mp9v0u708s/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

✅ Saved! View your model here: https://huggingface.co/defnotutkarsh/hindi-llama-3-slang-v1


In [ ]:
# Cell 4: Launch the Chat App (Shareable Link)
import locale
locale.getpreferredencoding = lambda: "UTF-8" # Fix for Colab locale bug
!pip install -q gradio

import gradio as gr
from unsloth import FastLanguageModel

# 1. Prepare Model for Chatting
FastLanguageModel.for_inference(model)

# 2. The Chat Function
def chat_with_slang_ai(user_input):
    # Format the prompt exactly like the training data
    prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}""".format(user_input, "", "")

    # Generate
    inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)

    # Decode and clean up
    response = tokenizer.batch_decode(outputs)[0]
    final_answer = response.split("### Response:")[-1].replace("<|end_of_text|>", "").strip()
    return final_answer

# 3. The User Interface
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🇮🇳 Hindi Slang AI (V2)")
    gr.Markdown("This AI has been fine-tuned for 300 steps to speak like a real 'Bhai'. Ask it anything!")

    with gr.Row():
        user_box = gr.Textbox(label="Apna Sawaal Pucho", placeholder="Bhai, coding start kaise karu?")

    output_box = gr.Textbox(label="AI Ka Jawaab")
    btn = gr.Button("Submit / Bhejo", variant="primary")

    btn.click(chat_with_slang_ai, inputs=user_box, outputs=output_box)

# 4. Launch!
print("🚀 Launching App...")
demo.launch(share=True)

/tmp/ipython-input-3872862320.py:36: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


🚀 Launching App...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b8ba2794b4f2fefe7e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
